# 프록시 디자인 패턴

## 목적

- 복잡한 시스템을 간단하게 표현
- 객체에 대한 보안 제공
- 다른 서버에 존재하는 외부 객체에 대한 로컬 인터페이스를 제공
- 메모리 사용량이 높은 객체를 다루는 가벼운 핸들러 역할

## 역할

- 특정 객체의 대리 객체를 제공해 접근 제어
- 분산 구조를 지원하기 위한 레이어 또는 인터페이스 제공
- 의도하지 않은 케이스로부터 객체 보호

### 구성 요소의 역할

- `Proxy`: 실 객체에 접근할 수 있는 레퍼런스 유지. `Subject`와 동일한 인터페이스 구조를 가지므로 실 객체를 대체할 수 있음. `RealSubject`의 생성과 소멸 담당
- `Subject`: `RealSubject`와 `Proxy`를 책임짐. `Proxy`와 `RealSubject`가 `Subject` 인터페이스를 구현하기 때문에 `RealSubject`를 `Proxy`로 대체 가능
- `RealSubject`: `Proxy`가 대체하는 실 객체를 나타냄

## 프록시의 유형

### 가상 프록시(Virtual Proxy)

- 인스턴스화 하기 무거운 객체의 플레이스 홀더 역할
- 웹사이트의 큰 이미지에 클라이언트가 객체를 처음 요청하거나 접근했을 때 실 객체를 생성

### 원격 프록시(Remote Proxy)

- 원격 서버나 다른 주소 공간에 존재하는 객체에 대한 로컬 인스턴스 생성
- 다수의 웹서버, DB서버, 작업 서버, 캐시 서버 등으로 구성된 앱의 모니터링 시스템 구성시, 원격 객체를 로컬에서 제어할 수 있는 원격 프록시 객체를 생성

### 보호 프록시(Protective Proxy)

- `RealSubject`의 중요한 부분에 대한 접근 제어
- 사용자의 인증과 허가를 담당하는 인증 서비스
- 웹사이트의 핵심 기능을 허가 받지 않은 에이전트로부터 보호

### 스마트 프록시(Smart Proxy)

- 사용자가 객체에 접근했을 때 추가 행동을 취함
- 상태를 중앙 서버에 저장하는 핵심 기능을 하는 서비스에서 리소스 공유 문제를 방지하기 위해 각 서비스가 이 기능을 직접 호출하는 대신 스마트 프록시가 객체의 잠금 상태를 확인하는 기능을 추가로 수행해 접근 제어

## 퍼사드와 프록시 패턴 비교

| 프록시 패턴                                                         | 퍼사드 패턴                                |
| ------------------------------------------------------------------- | ------------------------------------------ |
| 실 객체의 대리 객체를 제공해 접근 제어                              | 클래스의 서브시스템에 대한 인터페이스 제공 |
| 타겟 객체와 동일한 인터페이스 구조를 가지며 타겟에 대한 참조를 가짐 | 서브시스템 간의 의존도와 통신 최소화       |
| 클라이언트와 감싸고 있는 객체 사이의 중재자 역할                    | 하나의 통합된 간단한 인터페이스를 제공     |


In [1]:
# 프록시 디자인 패턴
class Actor:
    def __init__(self):
        self.isBusy = False

    def occupied(self):
        self.isBusy = True
        print(type(self).__name__, "is occupied with current movie")

    def available(self):
        self.isBusy = False
        print(type(self).__name__, "is free for the movie")

    def getStatus(self):
        return self.isBusy


class Agent:
    def __init__(self):
        self.principal = None

    def work(self):
        self.actor = Actor()
        if self.actor.getStatus():
            self.actor.occupied()
        else:
            self.actor.available()


if __name__ == "__main__":
    r = Agent()
    r.work()

# Actor is free for the movie

Actor is free for the movie


In [11]:
# 프록시 디자인 패턴 - 결제 시스템 예시
from abc import ABCMeta, abstractmethod


class Payment(metaclass=ABCMeta):
    @abstractmethod
    def do_pay(self):
        pass


class Bank(Payment):
    def __init__(self):
        self.card = None
        self.accout = None

    def __getAccount(self):
        self.accout = self.card
        return self.accout

    def __hasFunds(self):
        print("Bank:: Checking if Account", self.__getAccount(), "has enough funds")
        return True

    def setCard(self, card):
        self.card = card

    def do_pay(self):
        if self.__hasFunds():
            print("Bank:: Paying the merchant")
            return True
        else:
            print("Bank:; Sorry, not enouph funds!")
            return False


class DebitCard(Payment):
    def __init__(self):
        self.bank = Bank()

    def do_pay(self):
        card = input("Proxy:: Punch in Card Number: ")
        self.bank.setCard(card)
        return self.bank.do_pay()


class You:
    def __init__(self):
        print("You:: Lets buy the Denim shirt")
        self.debitCard = DebitCard()
        self.isPurchased = None

    def make_payment(self):
        self.isPurchased = self.debitCard.do_pay()
        self.print_result()

    def print_result(self):
        if self.isPurchased:
            print("You:: Wow! Denim shirt is Mine :-)")
        else:
            print("You:: I should earn more :(")


you = You()
you.make_payment()

You:: Lets buy the Denim shirt
Bank:: Checking if Account 1111 has enough funds
Bank:: Paying the merchant
You:: Wow! Denim shirt is Mine :-)
